In [201]:
import pandas as pd
import numpy as np
import json

#cargo los datasets
#si agrego  dtype='unicode' evito algunos errores sobre tipos de datos
dfMovies = pd.read_csv(r'datasets\movies_dataset.csv', dtype='unicode')
dfCredits= pd.read_csv(r'datasets\credits.csv', dtype='unicode')

pd.set_option("display.max_rows", 2000, "display.max_columns", None)

In [189]:
#TRANSFORMACIONES VARIAS ---> NO OLVIDAR CORRER LO QUE ESTA COMENTADO

#lleno NaNs con 0s en las columnas revenue y budget
#elimino rows que contienen un NaN en la columna release_date
#elimino columnas que no se utilizaran
try:
    dfMovies['revenue'] = dfMovies['revenue'].fillna(0)
    dfMovies['budget'] = dfMovies['budget'].fillna(0)
    dfMovies = dfMovies.dropna(subset='release_date')
    dfMovies= dfMovies.drop(columns=["video","adult","imdb_id" ,"original_title", "poster_path", "homepage"])
except:
    print("estas transformaciones ya se ejecutaron")

estas transformaciones ya se ejecutaron


In [192]:
#FUNCIONES

#desanida columna y devuelve un dataframe
def desanidarColumna(columna):
    lista = []
    listaDict=[]
    counter=0
    for k, v in columna.items():
        if isinstance(v, str):
            lista.append(v)
        else:
            lista.append(None)
    for i in lista:   
                try:
                    i=json.loads(lista[counter].replace("'", '"'))
                    listaDict.append(i)
                    counter+=1
                except:
                    listaDict.append(None)
                    counter+=1
    listaDict=pd.json_normalize(listaDict)
    return listaDict

def desanidarVariasColumnas(columna):
    lista=[]
    for col in columna:            
        #normalizo cada columna
        colAnidada=pd.json_normalize(columna[col])
        lista.append(colAnidada) #meto cada columna en una lista

        #armo un df concatenando cada columna de la lista
        nuevoDF=pd.concat(lista)
    return nuevoDF


In [202]:
#desanido belongs to collection

Títulos = (desanidarColumna(dfMovies['belongs_to_collection']))
#dfMovies["Títulos"] = Títulos["name"] es otra forma de agregar la columna
#agrego la columna al inicio
try:
    dfMovies.insert(0, "Títulos", Títulos["name"])
    dfMovies= dfMovies.drop(columns=["belongs_to_collection"])
except:
    print("Transformaciones ya fueron ejecutadas")

,Títulos,adult,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,Toy Story Collection,False,30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415
1,NaN,False,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413
2,Grumpy Old Men Collection,False,0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92
3,NaN,False,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34
4,Father of the Bride Collection,False,0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,NaN,False,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",http://www.imdb.com/title/tt6209470/,439050,tt6209470,fa,رگ خواب,Rising and falling between a man and woman.,0.072051,/jldsYflnId4tTWPx8es3uzsB1I8.jpg,[],"[{'iso_3166_1': 'IR', 'name': 'Iran'}]",NaN,0,90.0,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Released,Rising and falling between a man and woman,Subdue,False,4.0,1
45462,NaN,False,0,"[{'id': 18, 'name': 'Drama'}]",NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,0.178241,/xZkmxsNmYXJbKVsTRLLx3pqGHx7.jpg,"[{'name': 'Sine Olivia', 'id': 19653}]","[{'iso_3166_1': 'PH', 'name': 'Philippines'}]",2011-11-17,0,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,False,9.0,3
45463,NaN,False,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",0.903007,/d5bX92nDsISNhu3ZT69uHwmfCGw.jpg,"[{'name': 'American World Pictures', 'id': 6165}]","[{'iso_3166_1': 'US', 'name': 'United States o...",2003-08-01,0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,6
45464,NaN,False,0,[],NaN,227506,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",0.003503,

In [163]:
#la columna generos hay que desanidarle cada columna despues de aplciarle la funcion

dfGeneros = (desanidarColumna(dfMovies['genres'])) #le aplico la primera funcion
dfGeneros=desanidarVariasColumnas(dfGeneros) #desanido cada columna con la segunda funcion

dfGeneros

#de esta tabla solo necesito el GENERO de la pelicula odna ya estan ordenadas cada una con su genero
    #lo qeu tengoq ue hacer es literalmente extraer el genero y pegarlo en la columna genres, no hay que eliminar nulls
    #ni duplicados

#en teoria lo que tengo que hacer es agregar una columna que sea idGenero que tenga el id de la pelicula a la que tiene que
#apuntar, hay que cranear bien esta parte

,id,name
0,16.0,Animation
1,12.0,Adventure
2,10749.0,Romance
3,35.0,Comedy
4,35.0,Comedy
...,...,...
45461,NaN,NaN
45462,NaN,NaN
45463,NaN,NaN
45464,NaN,NaN


In [164]:
#desanido production_companies
#hay que revisar como manejo los duplicados, si esto es solo una tabla foranea tengo que eliminar
#todos los duplicados pero tengo que ver bien como resulta el uso de cada columna

dfCompanies=pd.DataFrame(dfMovies['production_companies'])
dfCompanies=desanidarColumna(dfMovies['production_companies'])
dfCompanies=desanidarVariasColumnas(dfCompanies)
dfCompanies

,name,id
0,Pixar Animation Studios,3.0
1,TriStar Pictures,559.0
2,Warner Bros.,6194.0
3,Twentieth Century Fox Film Corporation,306.0
4,Sandollar Productions,5842.0
...,...,...
44971,NaN,NaN
44972,NaN,NaN
44973,NaN,NaN
44974,NaN,NaN


In [165]:
#desanido production_countries
#hay que revisar como manejo los duplicados, si esto es solo una tabla foranea tengo que eliminar
#todos los duplicados pero tengo que ver bien como resulta el uso de cada columna

dfProdCountries=desanidarVariasColumnas(desanidarColumna(dfMovies["production_countries"]))
dfProdCountries



,iso_3166_1,name
0,US,United States of America
1,US,United States of America
2,US,United States of America
3,US,United States of America
4,US,United States of America
...,...,...
45453,NaN,NaN
45454,NaN,NaN
45455,NaN,NaN
45456,NaN,NaN


In [166]:
#desanido spoken_languages
#hay que revisar como manejo los duplicados, si esto es solo una tabla foranea tengo que eliminar
#todos los duplicados pero tengo que ver bien como resulta el uso de cada columna

dfSpokenLan=desanidarVariasColumnas(desanidarColumna(dfMovies["spoken_languages"]))
dfSpokenLan



,iso_639_1,name
0,en,English
1,en,English
2,en,English
3,en,English
4,en,English
...,...,...
45429,NaN,NaN
45430,NaN,NaN
45431,NaN,NaN
45432,NaN,NaN


In [171]:
#tablas que contienen fechas:
    #release_date
release_year = dfMovies["release_date"]
#cambio valores de fechas invalidas a fechas default
release_year[19730] = "1900-01-01"
release_year[29503] = "1900-01-01"
release_year[35587] = "1900-01-01"

#obtengo solo año de cada string de fecha y lo meto en release_year
dfMovies["release_date"] = pd.to_datetime(release_year).dt.year
dfMovies

#se supone que transforme la columna en date type pero es un int REVISAR


C:\Users\Administrator\AppData\Local\Temp\ipykernel_3200\3788619233.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  release_year[19730] = "1900-01-01"


,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1970,373554033,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,7.7,5415
1,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,en,When siblings Judy and Peter discover an encha...,17.015539,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1970,262797249,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413
2,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602,en,A family wedding reignites the ancient feud be...,11.7129,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1970,0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92
3,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1970,81452156,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34
4,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",11862,en,Just when George Banks has recovered from his ...,8.387519,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1970,76578911,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45460,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 28, 'name...",30840,en,"Yet another version of the classic epic, with ...",5.683753,"[{'name': 'Westdeutscher Rundfunk (WDR)', 'id'...","[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...",1970,0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Robin Hood,5.7,26
45462,NaN,0,"[{'id': 18, 'name': 'Drama'}]",111109,tl,An artist struggles to finish his work while a...,0.178241,"[{'name': 'Sine Olivia', 'id': 19653}]","[{'iso_3166_1': 'PH', 'name': 'Philippines'}]",1970,0,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,9.0,3
45463,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",67758,en,"When one of her hits goes wrong, a professiona...",0.903007,"[{'name': 'American World Pictures', 'id': 6165}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1970,0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,3.8,6
45464,NaN,0,[],227506,en,"In a small town live two brothers, one a minis...",0.003503,"[{'name': 'Yermoliev', 'id': 88753}]","[{'iso_3166_1': 'RU', 'name': 'Russia'}]",1970,0,87.0,[],Released,NaN,Satan Triumphant,0.0,0
